In [36]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

In [37]:
# Carregando os dados
dados = pd.read_csv('breast_cancer_diagnosis.csv')

#Juntando colunas categóricas separadas
dados['age_group'] = dados['Start Age'].astype('string') + '-' + dados['End Age'].astype('string')
dados['env_nodes_group'] = dados['Start_env_nodes'].astype('string') + '-' + dados['end_env_nodes'].astype('string')
dados['tumor_size'] = dados['Start tumor size'].astype('string') + '-' + dados['End tumor size'].astype('string')

del dados['End Age']
del dados['Start Age']
del dados['Start_env_nodes']
del dados['end_env_nodes']
del dados['Start tumor size']
del dados['End tumor size']

target = dados.pop('Class')
dados.insert(9,'Class',target)

#Ajustando os tipos de variáveis
dados['menopause'] = dados['menopause'].astype('category')
dados['menopause'].cat.categories = ['premeno', 'ge40', 'lt40']

dados['breast'] = dados['breast'].astype('category')
dados['breast'].cat.categories = ['right', 'left']

dados['node-caps'] = dados['node-caps'].astype('category')

dados['deg-malig'] = dados['deg-malig'].astype('category')
dados['deg-malig'].cat.categories = [1, 2, 3]

dados['breast-quad'] = dados['breast-quad'].astype('category')
dados['breast-quad'].cat.categories = ['left-up', 'left-low', 'right-up', 'right-low', 'central']

dados['irradiat'] = dados['irradiat'].astype('category')

dados['age_group'] = dados['age_group'].astype('category')
dados['age_group'].cat.categories = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']

dados['env_nodes_group'] = dados['env_nodes_group'].astype('category')
dados['env_nodes_group'].cat.categories = ['0-2', '3-5', '6-8', '9-11', '15-17', '18-20', '21-23']

dados['tumor_size'] = dados['tumor_size'].astype('category')
dados['tumor_size'].cat.categories = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54']

dados['Class'] = dados['Class'].astype('category')
dados['Class'].cat.categories = ['no-recurrence-events', 'recurrence-events']

#Melhorando a visualização da variável target
map_dictionary = {'recurrence-events' : "yes", 'no-recurrence-events' : "no"}
dados['Class'] = dados['Class'].map(map_dictionary)

dados.rename(columns = {'Class':'recurrent_events'}, inplace=True)

dados

,menopause,node-caps,deg-malig,breast,breast-quad,irradiat,age_group,env_nodes_group,tumor_size,recurrent_events
0,lt40,yes,3,left,right-up,no,40-49,0-2,10-14,yes
1,premeno,no,1,left,left-up,no,50-59,0-2,10-14,no
2,premeno,no,2,right,left-low,no,50-59,0-2,30-34,yes
3,lt40,yes,3,left,left-low,yes,40-49,0-2,30-34,no
4,lt40,yes,2,right,central,no,40-49,15-17,25-29,yes
...,...,...,...,...,...,...,...,...,...,...
281,premeno,yes,2,right,left-low,no,50-59,18-20,25-29,no
282,lt40,yes,2,right,left-low,yes,50-59,15-17,20-24,no
283,lt40,yes,2,left,central,no,30-39,18-20,25-29,no
284,lt40,no,2,left,left-low,no,50-59,0-2,10-14,no


In [40]:
#Ajustando as variaveis para valores numericos
dados['menopause'] = dados['menopause'].astype('category')
dados['menopause'].cat.categories = [0, 2, 1]

dados['breast'] = dados['breast'].astype('category')
dados['breast'].cat.categories = [0, 1]

dados['node-caps'] = dados['node-caps'].astype('category')
dados['node-caps'].cat.categories = [0,1]

dados['deg-malig'] = dados['deg-malig'].astype('category')
dados['deg-malig'].cat.categories = [1, 2, 3]

dados['irradiat'] = dados['irradiat'].astype('category')
dados['irradiat'].cat.categories = [0,1]

dados['age_group'] = dados['age_group'].astype('category')
dados['age_group'].cat.categories = ['25', '35', '45', '55', '65', '75']

dados['env_nodes_group'] = dados['env_nodes_group'].astype('category')
dados['env_nodes_group'].cat.categories = ['1', '4', '7', '10', '16', '19', '22']

dados['tumor_size'] = dados['tumor_size'].astype('category')
dados['tumor_size'].cat.categories = ['2', '7', '12', '17', '22', '27', '32', '37', '42', '47', '52']

map_dictionary = {'yes' : 1, 'no-' : 0}
dados['Class'] = dados['Class'].map(map_dictionary)

dados

KeyError: 'Class'

In [32]:
crosstabs = []
for coluna in dados:
    x = pd.crosstab(dados[coluna],dados['recurrent_events'])
    crosstabs.append(x)
crosstabs[0]

recurrent_events,0,1
menopause,,
0,94,35
2,5,2
1,102,48


In [33]:
features = ['menopause','node-caps','deg-malig','breast','irradiat','age_group','env_nodes_group','tumor_size']
X = dados[features]
y = dados.recurrent_events

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('Cancer.png')
Image(graph.create_png())

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').